<a href="https://colab.research.google.com/github/panportodos/first-repository/blob/master/20230515_Fine_Tune_GPT_3_For_Custom_Intent_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune GPT-3 For Custom Intent Classification

### 데이터 불러오기!

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = pd.read_csv("/content/watermelon_intents.csv", header=None)
data.head()

FileNotFoundError: ignored

위 코드는 pandas와 numpy라는 Python 라이브러리를 불러옵니다. 이 라이브러리들은 데이터를 쉽게 조작하고 분석하기 위해 사용됩니다. 그런 다음 np.random.seed(0)을 통해 무작위 숫자 생성의 시드를 설정합니다. 이렇게 하면 코드를 다시 실행해도 무작위 결과가 동일하게 유지됩니다! 마지막으로 pd.read_csv 함수를 사용해 CSV 파일을 불러오고 data.head()를 사용해 첫 5행을 출력합니다.

### 데이터 변환하기! 

유의사항 
1.  prompt와 completion이 끝나는 위치를 모델에 알리기 위해 각 prompt 및 completion 열에 구분 기호를 만듭니다. 
- 제 경우 prompt는 "\n\n->", completion은 " END"로 구분하지만 여러분이 원하시는 방법을 적용해도 무방합니다.
2. 데이터 프레임을 jsonl 파일로 변경합니다. [필수사항](https://platform.openai.com/docs/guides/fine-tuning/prepare-training-data).

In [ ]:
data.columns = ['prompt', 'completion']

In [ ]:
data['prompt'] = data['prompt'] + "\n\n-> "
data['completion'] = " "+data['completion'] + " END"
data.head(1)

In [ ]:
print(data['prompt'][0])

In [ ]:
data.to_json("watermelon.jsonl", orient='records', lines=True)

위 코드는 데이터를 모델이 이해할 수 있는 형식으로 변환합니다. 먼저 각 열에 'prompt'와 'completion'이라는 이름을 붙입니다. 그런 다음 각 프롬프트와 완성문에 구분자를 추가합니다. 이 구분자들은 모델에게 프롬프트와 완성문이 어디에서 시작하고 끝나는지 알려줍니다. 마지막으로 to_json 함수를 사용하여 데이터를 JSONL 형식의 파일로 저장합니다.

### Fine-tuning 하기!

In [ ]:
!pip install --upgrade openai

In [ ]:
!openai tools fine_tunes.prepare_data -f tampon.jsonl

In [ ]:
import os 
os.environ['OPENAI_API_KEY'] = "your api key insert here!"

In [ ]:
!openai api fine_tunes.create -t "tampon_prepared.jsonl" -m 'davinci'

위 코드는 OpenAI 라이브러리를 설치하고, 준비된 데이터를 fine-tuning 하기 위해 OpenAI 서버로 보냅니다. os.environ['OPENAI_API_KEY']은 OpenAI 계정의 API 키를 환경 변수로 설정합니다. 이 키는 OpenAI 서비스에 대한 액세스를 인증하는 데 사용됩니다. 마지막으로, openai api fine_tunes.create 명령을 통해 fine-tuning을 시작합니다. 이 과정은 OpenAI 서버에서 수행되므로 인터넷 연결이 끊어져도 괜찮습니다.

In [ ]:
# Fine-tuning은 OpenAI 서버에서 이루어지기 때문에 인터넷 연결이 끊겨도 괜찮습니다. 
# 아래 코드를 실행하면 Fine-tuning 진행경과를 확인할 수 있습니다. 

# !openai api fine_tunes.follow -i ft-R9vJN8hWHEQkQpxN0Xol2Z3Z



### 나만의 파인튜닝 모델 테스트 해보기

In [ ]:
prompt = "배송 빨랐구, 포장 짱짱해요~ 사이즈는 여름 수박에 비해 많이 작아요 그래도 달아서 잘 먹음 ㅎㅎ \n\n->"

In [ ]:
import openai
openai.api_key = 'your api key here! '
response = openai.Completion.create(
    model = "your fine-tuned model name here!",
    prompt=prompt,
    max_tokens=254,
    temperature=0,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=[" END"]
)
print(response['choices'][0]['text'])
# 예시 답변: delivery, packaging, size, tasty, satisfaction

위 코드 부분은 fine-tuned 모델을 테스트하는 과정입니다. 먼저, 테스트할 프롬프트를 정의합니다. 그 후, OpenAI 라이브러리를 import하고 API 키를 설정합니다.

openai.Completion.create는 API를 호출하여 특정 프롬프트에 대한 모델의 완성문을 생성합니다. 여기서는 여러 파라미터를 설정할 수 있습니다:

- model: 사용할 모델의 이름입니다. 여기에는 fine-tuned한 모델의 이름을 넣어야 합니다.
- prompt: 모델에게 제공할 프롬프트입니다. 이 프롬프트를 바탕으로 모델은 완성문을 생성합니다.
- max_tokens: 모델이 생성할 최대 토큰 수입니다.
- temperature: 출력의 다양성을 제어합니다. 높은 값(예: 1.0)은 출력이 더 다양하게 되고, 낮은 값(예: 0.0)은 출력이 더 결정론적이 되도록 합니다.
- top_p: 토큰 생성 시 고려할 확률 분포의 상위 부분입니다. 이 값이 1.0이면 모든 토큰이 고려되며, 이 값이 작아질수록 높은 확률의 토큰만 고려됩니다.
- frequency_penalty 및 presence_penalty: 이들은 각각 자주 등장하는 단어와 새롭게 등장하는 단어에 대한 페널티를 제어합니다.
- stop: 이 토큰이 등장하면 생성을 중단합니다. 여기서는 "END"로 설정되어 있습니다.

마지막으로, print(response['choices'][0]['text'])를 통해 모델이 생성한 완성문을 출력합니다.